# 10.3.3 Scrape Mars Data: The News

In [218]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [219]:
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/krishnakalisarkar/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


## Visit the NASA Mars News Site

In [220]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [221]:
# Parse the HTML
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [222]:
slide_elem.find('div', class_= 'content_title')

<div class="content_title">MOXIE Could Help Future Rockets Launch Off Mars</div>

In [223]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'MOXIE Could Help Future Rockets Launch Off Mars'

In [224]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel."

# 10.3.4 Scrape Mars Data: Featured Image

### Featured Images

In [225]:
# Visit URL
url = 'https://spaceimages-mars.com/'
browser.visit(url)

In [226]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [215]:
# browser.quit()

In [227]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [228]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [229]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

# 10.3.5 Scrape Mars Data: Mars Facts

In [101]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [30]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [181]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'
browser.visit(url)

In [182]:
# Parse the resulting html with soup
html = browser.html
link_soup = soup(html, 'html.parser')
type(link_soup)

bs4.BeautifulSoup

In [183]:
link = link_soup.find('a', class_='itemLink product-item')['href']
link

'cerberus.html'

In [184]:
image_url = f'https://marshemispheres.com/{link}'
image_url

'https://marshemispheres.com/cerberus.html'

In [185]:
# 2a. Create a list to hold the images and titles.
image_urls = []
results = link_soup.find_all('div', class_ = 'item')
for result in results:
    # Retreive image urls
    link = result.find('a', class_='itemLink product-item')['href']
   
    # Create image url by appending to the base url
    image_url = f'https://marshemispheres.com/{link}'
    image_urls.append(image_url)
    
    print('--------------------------------------------------------')
    print(image_url)

--------------------------------------------------------
https://marshemispheres.com/cerberus.html
--------------------------------------------------------
https://marshemispheres.com/schiaparelli.html
--------------------------------------------------------
https://marshemispheres.com/syrtis.html
--------------------------------------------------------
https://marshemispheres.com/valles.html


## Extract full size .jpg image

In [191]:
import time
# 2b. Create a list to hold the images and titles.Step 2
hemisphere_image_urls = []

for url in image_urls:
    hemispheres = {}
    # Visit the url using browser
    browser.visit(url)
    # Click on toggle open button
    button = browser.find_by_id("wide-image-toggle")
    button.click()
    # Time lapse
    time.sleep(1)
    # Parse the resulting html with soup
    html = browser.html
    img_soup = soup(html, 'html.parser')
    
    # 3. Write code to retrieve the image urls and titles for each hemisphere.
    # Retreiving titles
    title = img_soup.find('h2', class_= "title").text
    hemispheres["title"]= title
   
    # Retreiving rel image url
    rel_img = img_soup.find('img',class_= "wide-image").get('src')
    img_url = f'https://marshemispheres.com/{rel_img}'
    hemispheres["img_url"]= img_url
    hemisphere_image_urls.append(hemispheres)
    browser.back()
    
    print('------------------------------------------------------------------------------------------------------')
    print(title)
    print(img_url)


------------------------------------------------------------------------------------------------------
Cerberus Hemisphere Enhanced
https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg
------------------------------------------------------------------------------------------------------
Schiaparelli Hemisphere Enhanced
https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg
------------------------------------------------------------------------------------------------------
Syrtis Major Hemisphere Enhanced
https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg
------------------------------------------------------------------------------------------------------
Valles Marineris Hemisphere Enhanced
https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg


In [192]:
# 4. Print the list that holds the dictionary of each image url and title.
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]


In [196]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg')

In [197]:
Image(url= 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg')

In [198]:
Image(url= 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg')

In [199]:
Image(url= 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg')


In [200]:
# 5. Quit the browser
browser.quit()

## End of Deliverable: 1